In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/BatPopGenetics/Rhinolophus_affinis/7.Fstcalc/TableS1.Samples information_2023-08-01.xlsx')
data

,Sample_name,sn,Host_species,Sample_type,Province,City,Site,Longitude,Latitude,Collection_time,Raw_Base(Gb),Clean_Base(Gb),Clean_Read1_num,Clean_Read1_avg_len,Clean_Read2_avg_len
0,B100,R2109032424,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,30.40,14.670611,51000989,143.8,143.8
1,B101,R2109032425,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,40.84,8.081463,27792230,145.4,145.4
2,B102,R2109032426,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,44.26,19.282886,66686087,144.6,144.6
3,B103,R2109032427,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,36.50,15.505530,53661587,144.5,144.5
4,B104,R2109032428,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,54.46,21.030041,73139115,143.8,143.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,HL19,R2012010352,Pipistrellus abramus,Anal swab,Hainan,Qiongzhong,Site12_QZ,109.677432,18.997514,2008-07-03,48.26,9.266148,32886857,141.1,140.6
601,HL20,R2012010353,Pipistrellus abramus,Anal swab,Hainan,Qiongzhong,Site12_QZ,109.677432,18.997514,2008-07-03,52.42,10.586133,37188437,142.5,142.2
602,XL26,R2109032618,Hipposideros pomona,Anal swab,Hainan,Qiongzhong,Site13_QZ,109.751167,18.928437,2008-07-06,12.14,9.428727,33615176,140.3,140.2
603,XL31,R2109032619,Rhinolophus affinis,Anal swab,Hainan,Qiongzhong,Site13_QZ,109.751167,18.928437,2008-07-06,6.74,5.554121,20314893,136.7,136.7


In [3]:
data['Year'] = data['Collection_time'].apply(lambda x: x.year)

In [10]:
Ra_data = data.loc[(data['Host_species'] == 'Rhinolophus sinicus') & (data['Site'].isin(['Site1_GZ', 'Site6_HZ'])), ['sn', 'Site', 'Year']]
Ra_data['Site_Year'] = Ra_data['Site'] + '_' + Ra_data['Year'].astype(str)
Ra_data = Ra_data[Ra_data['Year']!= 2012]

In [11]:
def runcommand(command):
    import subprocess
    p = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    stdout, stderr = p.communicate()
    code = p.returncode
    return(code, stdout, stderr)


In [12]:
vcf = '/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/BatPopGenetics/Rhinolophus_sinicus/3.vcf_merge_QC/3.6.filtsampsite.vcf'

In [13]:
for line in open(vcf).readlines():
	if line.startswith('#CHROM'):
		vcfdfheader = line.lstrip('#').split()
		break
vcf_samples = set(vcfdfheader[9:])

In [14]:
fullresult = []
for (site, year), groupdf in Ra_data.groupby(['Site', 'Year']):
    open(f'{site}_{year}.snlist', 'w').write('\n'.join(groupdf['sn']))
    code, stdout, stderr = runcommand('vcftools --vcf ../3.vcf_merge_QC/3.6.filtsampsite.vcf --keep {group}.snlist --window-pi 1000 --out {group}'.format(group = site + '_' + str(year)))
    if code:
        print(f'{site}_{year} non-zero exit!')
        break
    else:
        group_pi_result = pd.read_table(f"{site}_{year}.windowed.pi")
        group_pi_result['site'] = site
        group_pi_result['year'] = year
        fullresult.append(group_pi_result)
        print(f"{site}_{year} calc done!")

Site1_GZ_2020 calc done!
Site1_GZ_2021 calc done!
Site6_HZ_2013 calc done!
Site6_HZ_2020 calc done!
Site6_HZ_2021 calc done!


In [15]:
pi_result_full = pd.concat(fullresult)
pi_result_full['host_species'] = "Rhinolophus sinicus"
pi_result_full.to_csv('Rs_host_windowedpi.csv', index=False)